The next cell will authenticate your bot based on your keys and tokens. See the [Tweepy tutorial](https://github.com/comp-journalism/UMD-J479V-J779V-Spring2016/blob/master/Weekly/Week_5/Tweepy-skeleton.ipynb) for more details. 

# Bot Skeleton

In [ ]:
import tweepy
import urllib
import time
#Setup and authenticate Tweepy
CONSUMER_KEY = "Your_key"
CONSUMER_SECRET = "Your_Secret"
ACCESS_TOKEN = "TOKEN"
ACCESS_TOKEN_SECRET = "Secret"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

me = api.me()
print me.screen_name



Terps_Weather


The next cell has a number of essential functions that allow this bot to tweet weather updates.

In [ ]:
import requests
import json

while True:
    j = 0; 
    while j < 29:
        cities = ['Aberdeen','Annapolis', 'Baltimore', 'Bowie', 'Brunswick', 'Cambridge', 'College Park', 'Crisfield', 'Cumberland',
                  'District Heights', 'Frederick', 'Frostbug', 'Fruitland', 'Gaithersburg', 'Glenarden', 'Greenbelt', 'Hagerstown',
                  'Havre De Grace','Hyattsville', 'Laurel', 'Mount Rainier', 'New Carrollton', 'Pocomoke City', 'Rockville',
                  'Salisbury', 'Seat Pleasant', 'Takoma Park','Taneytown', 'Westminster']
        # Making a GET request to weather underground API. This request is made for forecast feature of this API.
        # This returns a JSON object with forecast for the next 3 days(including today)

        url = 'http://api.wunderground.com/api/8f22a9ece2e9226f/geolookup/forecast/q/MD/' + cities[j] + '.json' 
        try:
            f = requests.get(url)

        except Exception as e:
            print e
            pass

        # parsing the data into readable JSON
        parsed_json = f.json()
        
        # Checking if there was any error key in response. If there is, displaying the error description and exiting the script. 
        if 'error' in parsed_json['response']:
            print parsed_json['response']['error']['description']
            exit()

        # Defining a function where it reads & stores the text for tweet from forecast key present in JSON object. It also reads the day
        # from the JSON object. The JSON object contains weather forecast for day and night in each response object.
        # Calling checkChar function from this function
        def forecast(i):
            if i == 0:
                first_day = parsed_json['forecast']['txt_forecast']['forecastday'][i]['title']
                tweet = "Today" + '(' + first_day + ')\n' + parsed_json['forecast']['txt_forecast']['forecastday'][i]['fcttext']

            if i == 1:
                first_day_night = parsed_json['forecast']['txt_forecast']['forecastday'][i]['title']
                tweet = "Tonight" + '(' + first_day_night + ')\n' + parsed_json['forecast']['txt_forecast']['forecastday'][i]['fcttext']
                                
            checkChar(tweet)
            return
        # This function splits the tweet in two parts. Twitter has a char limit of 140 however the limit has been set to 100 because 
        # it also counts image link in char limit(23) and the algorithm also appends city name. Returns the split variables as an output
        def splitChar(value):
            split1 = value[:100]
            split2 = value[100:200]
            return split1,split2
        
        # Checks for the length of tweet text parsed from forecast(). Modifies the text of these two tweets by adding 1/2 and 2/2 showing
        # continuation of tweet. Also tweets an appropriate image according to the current weather
        def checkChar(value):
            # Setting the path as per local machine. Change this path as per your system's file location.
            path = 'C:/Users/KrutikaSatish/Desktop/images/'
            if len(value) > 100:
                var1,var2 = splitChar(value)
                var1 = cities[j] + ' ' + var1 + '..1/2'
                var2 = var2 + '..2/2'
                # Fetching the name of the appropriate image from the response The images were downloaded from Weather Underground API.
                # from here http://www.wunderground.com/weather/api/d/docs?d=resources/icon-sets&MR=1.
                img = path + parsed_json['forecast']['txt_forecast']['forecastday'][i]['icon'] + '.gif'
                #Updating the tweet with media content. Tweeting the image only once in the second part of the tweet.
                api.update_status(var1)
                api.update_with_media(img, status=var2)
                
            else:
                # Doing the same logic when the tweet is below the char limit. In this case just tweeting the city name and 
                # posting tweet with appropriate media
                img = path + parsed_json['forecast']['txt_forecast']['forecastday'][i]['icon'] + '.gif'
                value = cities[j] + ' ' + value
                api.update_with_media(img, status=value)
                

        # Logic for tweeting forecast for day and night. Calling forecast() once for each
        i = 0  
        while i < 2:
            forecast(i)
            i += 1
        # Incrementing city array index and putting the script to sleep for 2979 seconds. Timeline has been kept as 2979 seconds because
        # twitter doesnt allow posting same tweet(I am including 29 cities and we have 24 hours in a day, hence 3600x24/29 
        # (no of seconds in a day/ no of cities)).
        # Therefore, this script will run for each city once a day. Once the day is over,
        # the weather will change and so will the text of the tweet.
        j += 1
        time.sleep(2979)